In [1]:
import pandas as pd

df_10_13 = pd.read_csv('data/cleaned_projects_raw_10_13.csv')
df_14 = pd.read_csv('data/cleaned_projects_raw_14.csv')
df_19 = pd.read_csv('data/cleaned_projects_raw.csv')

In [3]:
df_10_14 = pd.concat([df_10_13, df_14], axis = 0, ignore_index=True)
df_11_19 = pd.concat([df_10_14, df_19], axis = 0, ignore_index=True)
print(df_11_19.shape)

(227633, 16)


In [4]:
df_11_19.to_csv('data/cleaned_projects_raw_11_19.csv')

In [ ]:
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
import joblib
import os
import nltk
# nltk.download('punkt')      # Normal Sentence Segmentation Model
# nltk.download('punkt_tab')  

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [6]:
# set a new copy
df = df_11_19.copy()
# keep only the non-empty text
df = df[df['description'].notna()]

In [7]:
# excessive spaces and special characters
df['description'] = df['description'].str.replace(r'\s+', ' ', regex=True).str.strip()

In [8]:
# split
df['sentences'] = df['description'].apply(sent_tokenize)

In [9]:
# flatten all the sentences
all_sentences = df['sentences'].explode().tolist()

In [10]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [11]:
# encode all sentences (with caching)
BATCH_SIZE = 64
CACHE_FILE = 'sbert_vectors.pkl'

if os.path.exists(CACHE_FILE):
    print("Loading cached embeddings...")
    sentence_embeddings = joblib.load(CACHE_FILE)
else:
    print("Encoding...")
    sentence_embeddings = model.encode(
        all_sentences,
        batch_size=BATCH_SIZE,
        show_progress_bar=True
    )
    joblib.dump(sentence_embeddings, CACHE_FILE)
    print("Embeddings cached!")

Loading cached embeddings...


In [12]:
emb_idx = 0
final_vectors = []

for sent_list in tqdm(df['sentences']):
    count = len(sent_list)
    if count == 0:
        final_vectors.append(np.zeros(model.get_sentence_embedding_dimension()))
    else:
        vecs = sentence_embeddings[emb_idx: emb_idx + count]
        final_vectors.append(np.mean(vecs, axis=0))
        emb_idx += count

 81%|████████▏ | 184878/227508 [00:01<00:00, 129071.40it/s]e:\dissertation\project\housing_feature_gentrification\venv\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
e:\dissertation\project\housing_feature_gentrification\venv\Lib\site-packages\numpy\_core\_methods.py:137: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
100%|██████████| 227508/227508 [00:01<00:00, 120271.78it/s]


In [13]:
# DataFrame + merge back to main list
vec_df = pd.DataFrame(final_vectors)
vec_df.columns = [f'sbert_{i}' for i in range(vec_df.shape[1])]
df_vectors = pd.concat([df.reset_index(drop=True), vec_df], axis=1)

In [14]:
# save a copy
df_vectors.to_parquet('sbert_encoded_data_11_19.parquet')

In [19]:
print(df_vectors.head())

          site_name decision_date  valid_date  \
0               NaN    2010-05-02  2010-03-08   
1  Development Site    2011-03-31  2010-11-30   
2               NaN           NaN  2010-04-09   
3                38    2011-01-07  2010-11-15   
4               NaN    2011-04-28  2010-06-16   

                                  polygon.geometries        polygon.type  \
0  [{'coordinates': [[[529321.6973703, 187356.898...  GeometryCollection   
1  [{'coordinates': [[[544084.439, 183412.144], [...  GeometryCollection   
2                                                NaN                 NaN   
3                                                NaN                 NaN   
4                                                NaN                 NaN   

                           wgs84_polygon.coordinates wgs84_polygon.type  \
0  [[[-0.1353955, 51.5702629], [-0.1352572, 51.57...            Polygon   
1  [[[0.075859, 51.5312398], [0.075848, 51.531291...            Polygon   
2  [[[-0.087185, 51.544